In [6]:
import os


In [7]:
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\DataScienceProject\\research'

In [8]:
os.chdir("../")
%pwd

'c:\\Users\\t0263519\\Desktop\\Study\\PYTHON\\MLFLOW-Projects\\DataScienceProject'

In [9]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
#from src.datascience.constants import *
from src.DataScience.utils.common import *



ModuleNotFoundError: No module named 'src.DataScience'

In [3]:

class ConfigurationManager():
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config


ModuleNotFoundError: No module named 'src.DataScience'